In [29]:
!pip install spacy
!python -m spacy download en_core_web_sm



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/shivakumarkaranam/Documents/infosys/infosys/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [30]:
import torch
import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from peft import PeftModel

# Load spaCy for aspect extraction
nlp = spacy.load("en_core_web_sm")

# Your fine-tuned LoRA model
MODEL_ID = "Shiva-k22/sentiment-lora-distilbert"
BASE_MODEL = "distilbert-base-uncased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=3)
model = PeftModel.from_pretrained(base_model, MODEL_ID)

# Define label mapping
LABEL_MAP = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

# Create pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
/Users/shivakumarkaranam/Documents/infosys/infosys/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [31]:
# Extract aspects using spaCy
def extract_aspects(text):
    """Extract candidate aspects using spaCy noun phrases + named entities."""
    doc = nlp(text)
    aspects = set()
    for chunk in doc.noun_chunks:
        aspects.add(chunk.text.strip())
    for ent in doc.ents:
        aspects.add(ent.text.strip())
    return list(aspects)


In [40]:
def aspect_sentiment_analysis(text):
    aspects = extract_aspects(text)
    results = []
    for aspect in aspects:
        prompt = f"Aspect: {aspect}. Sentence: {text}"
        output = pipe(prompt)[0]
        label = LABEL_MAP.get(output["label"], output["label"])
        results.append({
            "aspect": aspect,
            "sentiment": label,
            "score": round(float(output['score']), 3)
        })
    return results


In [41]:
sentences = [
    "Battery life is good but camera is disappointing.",
    "Shell's $70 Billion BG deal meets shareholder skepticism.",
    "$MSFT SQL Server revenue grew double-digit.",
    "The bank posted a net profit of 8.2 million euros."
]

for s in sentences:
    print("\nSentence:", s)
    for res in aspect_sentiment_analysis(s):
        print(f" → {res['aspect']} → {res['sentiment']} ({res['score']})")



Sentence: Battery life is good but camera is disappointing.
 → camera → Neutral (0.674)
 → Battery life → Positive (0.524)

Sentence: Shell's $70 Billion BG deal meets shareholder skepticism.
 → Shell's $70 Billion BG deal → Negative (0.563)
 → shareholder skepticism → Neutral (0.643)
 → Shell → Neutral (0.799)
 → $70 Billion → Negative (0.543)

Sentence: $MSFT SQL Server revenue grew double-digit.
 → SQL Server revenue → Neutral (0.445)
 → SQL Server → Neutral (0.518)
 → $MSFT → Negative (0.574)

Sentence: The bank posted a net profit of 8.2 million euros.
 → The bank → Neutral (0.98)
 → 8.2 million → Neutral (0.951)
 → a net profit → Neutral (0.954)
 → 8.2 million euros → Neutral (0.988)


In [42]:
import pandas as pd
dataset = pd.read_csv("/Users/shivakumarkaranam/Documents/infosys/cleaned_sentiment_data.csv")

In [43]:
for i in range(10):
    text = dataset['Cleaned_Sentence'][i]
    print("\nSentence:", text)
    for res in aspect_sentiment_analysis(text):
        print(f" → {res['aspect']} → {res['sentiment']} ({res['score']})")
        


Sentence: geosolutions technology leverage benefon gps solution provide location base search technology community platform location relevant multimedia content new powerful commercial model .
 → solution → Neutral (0.777)
 → geosolutions technology leverage benefon → Neutral (0.646)
 → location base search technology community platform location relevant multimedia content new powerful commercial model → Neutral (0.712)

Sentence: $ esi low $ 1.50 $ 2.50 bk real possibility
 → real possibility → Negative (0.649)
 → $ esi low → Negative (0.888)
 → 2.50 → Negative (0.839)

Sentence: last quarter 2010 componenta net sale double eur131m eur76m period year earlier move zero pretax profit pretax loss eur7m .
 → m → Neutral (0.695)
 → period year earlier → Positive (0.547)
 → eur76 → Positive (0.628)
 → double eur131m eur76 → Positive (0.917)
 → eur7 → Positive (0.53)
 → last quarter 2010 → Positive (0.679)
 → zero pretax profit pretax loss → Neutral (0.643)
 → zero → Neutral (0.768)
 → last 

In [44]:
def analyze_aspects_with_overall(text):
    aspects = extract_aspects(text)
    results = []

    # 🌟 1️⃣ Predict Overall Sentiment
    overall_res = pipe(text)[0]
    overall_label = LABEL_MAP.get(overall_res["label"], overall_res["label"])
    overall_score = round(float(overall_res["score"]), 3)

    # 🌟 2️⃣ Predict Aspect-Level Sentiment
    for aspect in aspects:
        prompt = f"Aspect: {aspect}. Sentence: {text}"
        res = pipe(prompt)[0]
        label = LABEL_MAP.get(res["label"], res["label"])
        score = round(float(res["score"]), 3)
        results.append({
            "aspect": aspect,
            "aspect_sentiment": label,
            "aspect_score": score
        })

    # 🌟 Combine Results
    return {
        "sentence": text,
        "overall_sentiment": overall_label,
        "overall_score": overall_score,
        "aspects": results
    }



In [49]:

sentences = [
    "Battery life is good but camera is disappointing.",
]

results = [analyze_aspects_with_overall(s) for s in sentences]




In [50]:

# Convert to DataFrame

rows = []
for r in results:
    for a in r["aspects"]:
        rows.append({
            "Sentence": r["sentence"],
            "Aspect": a["aspect"],
            "Aspect Sentiment": a["aspect_sentiment"],
            "Aspect Score": a["aspect_score"],
            "Overall Sentiment": r["overall_sentiment"],
            "Overall Score": r["overall_score"]
        })

df = pd.DataFrame(rows)
print(df)

                                            Sentence        Aspect  \
0  Battery life is good but camera is disappointing.        camera   
1  Battery life is good but camera is disappointing.  Battery life   

  Aspect Sentiment  Aspect Score Overall Sentiment  Overall Score  
0          Neutral         0.674           Neutral          0.747  
1         Positive         0.524           Neutral          0.747  
